In [ ]:
!pip install torch torchvision scikit-learn tqdm

In [ ]:
import zipfile
import os

zip_file_path = '/content/dataset_split.zip'
output_dir = '/content/'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_dir)

print(f"File '{zip_file_path}' unzipped to '{output_dir}'")
print(os.listdir(output_dir))

File '/content/dataset_split.zip' unzipped to '/content/'
['.config', 'dataset_split.zip', 'dataset_split', 'sample_data']


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tqdm import tqdm
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


In [ ]:
DATA_DIR = "dataset_split/mouth"

In [ ]:
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

val_test_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [ ]:
train_ds = datasets.ImageFolder(f"{DATA_DIR}/train", transform=train_transforms)
val_ds   = datasets.ImageFolder(f"{DATA_DIR}/val", transform=val_test_transforms)
test_ds  = datasets.ImageFolder(f"{DATA_DIR}/test", transform=val_test_transforms)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_ds, batch_size=32, shuffle=False)

print("Classes:", train_ds.classes)
print("Train:", len(train_ds), "Val:", len(val_ds), "Test:", len(test_ds))

Classes: ['no_yawn', 'yawn']
Train: 1013 Val: 216 Test: 219


In [ ]:
model = models.mobilenet_v3_large(weights="IMAGENET1K_V1")

Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth


100%|██████████| 21.1M/21.1M [00:00<00:00, 151MB/s]


In [ ]:
model.classifier[3] = nn.Linear(
    model.classifier[3].in_features, 1
)

model = model.to(device)

In [ ]:
for param in model.features.parameters():
    param.requires_grad = False

In [ ]:
criterion = nn.BCEWithLogitsLoss()

optimizer = optim.Adam(
    model.classifier.parameters(),
    lr=1e-4
)

In [ ]:
def train_one_epoch(model, loader):
    model.train()
    total_loss = 0

    for images, labels in tqdm(loader):
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(loader)

In [ ]:
def evaluate(model, loader):
    model.eval()
    y_true, y_pred, y_prob = [], [], []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)

            probs = torch.sigmoid(outputs).cpu().numpy()
            preds = (probs > 0.5).astype(int)

            y_true.extend(labels.numpy())
            y_pred.extend(preds)
            y_prob.extend(probs)

    return np.array(y_true), np.array(y_pred), np.array(y_prob)

In [ ]:
EPOCHS = 20

for epoch in range(EPOCHS):
    loss = train_one_epoch(model, train_loader)
    y_true, y_pred, _ = evaluate(model, val_loader)

    print(f"\nEpoch {epoch+1}/{EPOCHS}")
    print("Train Loss:", round(loss, 4))
    print(classification_report(y_true, y_pred, target_names=train_ds.classes))

100%|██████████| 32/32 [01:08<00:00,  2.13s/it]



Epoch 1/20
Train Loss: 0.6346
              precision    recall  f1-score   support

     no_yawn       0.63      0.99      0.77       108
        yawn       0.98      0.41      0.58       108

    accuracy                           0.70       216
   macro avg       0.80      0.70      0.67       216
weighted avg       0.80      0.70      0.67       216



100%|██████████| 32/32 [01:02<00:00,  1.95s/it]



Epoch 2/20
Train Loss: 0.5244
              precision    recall  f1-score   support

     no_yawn       0.65      0.82      0.73       108
        yawn       0.76      0.56      0.65       108

    accuracy                           0.69       216
   macro avg       0.71      0.69      0.69       216
weighted avg       0.71      0.69      0.69       216



100%|██████████| 32/32 [00:59<00:00,  1.87s/it]


...


Epoch 18/20
Train Loss: 0.2387
              precision    recall  f1-score   support

     no_yawn       0.94      0.95      0.94       108
        yawn       0.95      0.94      0.94       108

    accuracy                           0.94       216
   macro avg       0.94      0.94      0.94       216
weighted avg       0.94      0.94      0.94       216



100%|██████████| 32/32 [01:00<00:00,  1.88s/it]



Epoch 19/20
Train Loss: 0.2141
              precision    recall  f1-score   support

     no_yawn       0.95      0.91      0.93       108
        yawn       0.91      0.95      0.93       108

    accuracy                           0.93       216
   macro avg       0.93      0.93      0.93       216
weighted avg       0.93      0.93      0.93       216



100%|██████████| 32/32 [00:59<00:00,  1.87s/it]



Epoch 20/20
Train Loss: 0.2199
              precision    recall  f1-score   support

     no_yawn       0.80      0.98      0.88       108
        yawn       0.98      0.76      0.85       108

    accuracy                           0.87       216
   macro avg       0.89      0.87      0.87       216
weighted avg       0.89      0.87      0.87       216



In [ ]:
for param in model.features[-2:].parameters():
    param.requires_grad = True

optimizer = optim.Adam(model.parameters(), lr=1e-5)

In [ ]:
for epoch in range(8):
    loss = train_one_epoch(model, train_loader)
    y_true, y_pred, _ = evaluate(model, val_loader)

    print(f"\nFine-Tune Epoch {epoch+1}")
    print("Loss:", round(loss, 4))
    print(classification_report(y_true, y_pred))

100%|██████████| 32/32 [01:06<00:00,  2.09s/it]



Fine-Tune Epoch 1
Loss: 0.2239
              precision    recall  f1-score   support

           0       0.93      0.96      0.95       108
           1       0.96      0.93      0.94       108

    accuracy                           0.94       216
   macro avg       0.95      0.94      0.94       216
weighted avg       0.95      0.94      0.94       216



100%|██████████| 32/32 [01:07<00:00,  2.10s/it]



Fine-Tune Epoch 2
Loss: 0.2097
              precision    recall  f1-score   support

           0       0.93      0.95      0.94       108
           1       0.95      0.93      0.94       108

    accuracy                           0.94       216
   macro avg       0.94      0.94      0.94       216
weighted avg       0.94      0.94      0.94       216



100%|██████████| 32/32 [01:06<00:00,  2.09s/it]


...


Fine-Tune Epoch 6
Loss: 0.1814
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       108
           1       0.95      0.95      0.95       108

    accuracy                           0.95       216
   macro avg       0.95      0.95      0.95       216
weighted avg       0.95      0.95      0.95       216



100%|██████████| 32/32 [01:06<00:00,  2.08s/it]



Fine-Tune Epoch 7
Loss: 0.1509
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       108
           1       0.95      0.95      0.95       108

    accuracy                           0.95       216
   macro avg       0.95      0.95      0.95       216
weighted avg       0.95      0.95      0.95       216



100%|██████████| 32/32 [01:06<00:00,  2.08s/it]



Fine-Tune Epoch 8
Loss: 0.1705
              precision    recall  f1-score   support

           0       0.95      0.96      0.96       108
           1       0.96      0.95      0.96       108

    accuracy                           0.96       216
   macro avg       0.96      0.96      0.96       216
weighted avg       0.96      0.96      0.96       216



In [ ]:
y_true, y_pred, y_prob = evaluate(model, test_loader)

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=train_ds.classes))

print("ROC-AUC:", roc_auc_score(y_true, y_prob))


Confusion Matrix:
[[102   8]
 [  3 106]]

Classification Report:
              precision    recall  f1-score   support

     no_yawn       0.97      0.93      0.95       110
        yawn       0.93      0.97      0.95       109

    accuracy                           0.95       219
   macro avg       0.95      0.95      0.95       219
weighted avg       0.95      0.95      0.95       219

ROC-AUC: 0.9924937447873228


In [ ]:
torch.save(
    model.state_dict(),
    "mobilenetv3_mouth_state.pth"
)

print("MobileNetV3 mouth-state model saved!")

MobileNetV3 mouth-state model saved!
